# Cubeviz Demonstration Notebook

This notebook demonstrates the Cubeviz API in the Notebook setting. UI equivalents for these actions, as well as additional documentation about Cubeviz, can be found here: https://jdaviz.readthedocs.io/en/latest/cubeviz/

Import modules needed for this notebook.

In [1]:
import warnings

from photutils.aperture import CircularAperture
from regions import (PixCoord, CirclePixelRegion, CircleSkyRegion, EllipsePixelRegion,
                     EllipseSkyRegion)

from jdaviz import Cubeviz

We create a Cubeviz instance and show it.

In [2]:
cubeviz = Cubeviz()
cubeviz.show()

Application(config='cubeviz', docs_link='https://jdaviz.readthedocs.io/en/latest/cubeviz/index.html', events=[…

Now we load observations. If you already have the files on your local machine, you can 
load them by passing their file paths to `load_data` as strings. For this example, 
we will retrieve remote data from [MAST](https://mast.stsci.edu/) via `astroquery`
by passing the observation's URI as a string. By default, the downloaded files are 
saved to the current working directory. If you set `cache=False` in the `load_data` 
call, it will re-download the file if desired.

One other thing to note about retrieving MAST data through astroquery is that it caches
the data by default. It is possible for files to be updated in MAST with more recent calibrations
but remain the same size, in which case your local cached file would not be replaced by the new
version. You can turn off caching by setting `cache=False` in the `download_file` call to
force it to re-download the file if desired.

In [3]:
uri = "mast:JWST/product/jw02732-c1001_t004_miri_ch1-short_s3d.fits"

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    cubeviz.load_data(uri, cache=True)

INFO: Found cached file ./jw02732-c1001_t004_miri_ch1-short_s3d.fits with expected size 34165440. [astroquery.query]
!!!!!!!!!!!!!!!!!!! 1: SCI


In [4]:
import astropy.units as u
from specutils import SpectralRegion
subset_plugin = cubeviz.plugins['Subset Tools']
unit = u.um
subset_plugin.import_region([
    EllipsePixelRegion(center=PixCoord(x=15, y=15), width=9, height=8),  # Subset 1 (spatial)
    SpectralRegion(5 * unit, 5.3 * unit),  # Subset 2 (spectral)
], combination_mode="new")
spatial_with_spec = cubeviz.get_data(data_label="Spectrum (Subset 1, sum)",
                                            spectral_subset="Subset 2")

$$$$  Subset 2 None Subset 2 Spectrum (Subset 1, sum) <class 'specutils.spectra.spectrum.Spectrum'> {'statistic': None, 'spectral_axis_index': 0}
####  Subset: Subset 2 (data: Spectrum (Subset 1, sum))


/home/javer/projects/jdaviz/jdaviz/core/helpers.py:686: UserWarning: Not able to get Spectrum (Subset 1, sum) returned with subset Subset 2 applied of type <class 'specutils.spectra.spectrum.Spectrum'>. Exception: 
  warnings.warn(f"Not able to get {data_label} returned with"


UnboundLocalError: cannot access local variable 'spec_subset' where it is not associated with a value

In [5]:
from glue.config import data_translator
from specutils import Spectrum
handler, _ = data_translator.get_handler_for(Spectrum)
data_label = 'Spectrum (Subset 1, sum)'
mask_subset = 'Subset 2'
real_spectral = [sub for subsets in cubeviz.app.data_collection.subset_groups
                 for sub in subsets.subsets
                 if sub.data.label == data_label and subsets.label == mask_subset][0] # noqa
print(dir(real_spectral), real_spectral.pixel_component_ids)

print(cubeviz.app.data_collection[data_label])

spec_subset = handler.to_object(real_spectral, **{'statistic': None, 'spectral_axis_index': 0})

['__and__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gluestate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__invert__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setgluestate__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '__xor__', '_broadcasting', '_uuid', 'attributes', 'broadcast', 'component_ids', 'components', 'coordinate_components', 'data', 'delete', 'derived_components', 'do_broadcast', 'group', 'hub', 'label', 'main_components', 'ndim', 'paste', 'pixel_component_ids', 'primary_components', 'read_mask', 'register', 'shape', 'size', 'state_as_mask', 'style', 'subset_group', 'subset_state', 'to_index_list', 'to_mask', 'uuid', 'verbose_label', 'visible', 'visible_components', 'world_component_ids', 'write

IncompatibleAttribute: 

In [7]:
data = cubeviz.app.data_collection[data_label]
print('data components: ', data._components)
print('\ndata pixel components: ', data.pixel_component_ids)
print('\nsubset components: ', real_spectral.components)
# print(data.attributes)
print('\ndata external components: ', data._externally_derivable_components)
print('\nsubset att: ', real_spectral.subset_state.att)
print('\ndata meta: ', data.meta)
# print('\noriginal data wcs: ', cubeviz.app.data_collection[0].meta['_orig_spec'].wcs.spectral)
print('\nupdated data wcs: ', cubeviz.app.data_collection[0].coords)




data.get_data(real_spectral.subset_state.att)
# data.get_mask(real_spectral.subset_state)
# real_spectral.subset_state.to_mask(data)

data components:  OrderedDict({Pixel Axis 0 [x]: <glue.core.component.CoordinateComponent object at 0x74fd5b4aef90>, World 0: <glue.core.component.CoordinateComponent object at 0x74fd59a90590>, flux: <glue.core.component.Component object at 0x74fd64ead850>, uncertainty: <glue.core.component.Component object at 0x74fd64eadf50>, mask: <glue.core.component.Component object at 0x74fd5b5c7e30>})

data pixel components:  [Pixel Axis 0 [x]]

subset components:  [Pixel Axis 0 [x], World 0, flux, uncertainty, mask]

data external components:  OrderedDict()

subset att:  World 0

data meta:  OrderedDict({'uncertainty_type': 'std', 'spectral_axis_index': 0, '_pixel_scale_factor': 3.97217570860291e-13, '_default_color': '#e31a1c', 'Plugin': '3D Spectral Extraction', '_update_live_plugin_results': {'bg_spec_per_spaxel': False, 'dataset': 'jw02732-c1001_t004_miri_ch1-short_s3d', 'wavelength_dependent': False, 'background': 'None', 'bg_spec_add_results': {'label': 'background-spectrum', 'auto': True,

IncompatibleAttribute: World 0

In [12]:
cubeviz.app.get_subsets()

{'Subset 1': [{'name': 'EllipticalROI',
   'glue_state': 'RoiSubsetState',
   'region': <EllipsePixelRegion(center=PixCoord(x=15.0, y=15.0), width=9.0, height=8.0, angle=0.0 rad)>,
   'sky_region': None,
   'subset_state': <glue.core.subset.RoiSubsetState at 0x7525ed8db490>}],
 'Subset 2': Spectral Region, 1 sub-regions:
   (4.62440061e-07 um, 4.62520112e-07 um) }

Alternatively, the data and the configuration can be autodetected and loaded simultaneously by calling `jdaviz.open(f'{data_dir}/{fn}')`

The upper left viewer containing the FLUX cube is the default viewer that is accessible as follows.

In [ ]:
viewer = cubeviz.default_viewer

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to Astropy regions. Click on an image viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = cubeviz.plugins['Subset Tools'].get_regions(region_type="spatial")

In [ ]:
regions

It is also possible to programmatically pass a `regions` shape or a `photutils` aperture shape.

*NOTE: Sky regions are not yet supported for data cubes.*

In [ ]:
data = cubeviz.get_data('jw02732-o004_t004_miri_ch1-short_s3d[SCI]')

In [ ]:
# photutils aperture
my_aper = CircularAperture((15, 10), r=5)

# regions shape
my_reg = CirclePixelRegion(center=PixCoord(x=35, y=35), radius=10)

my_regions = [my_aper, my_reg]
cubeviz.plugins['Subset Tools'].import_region(my_regions, combination_mode='new')

To get the spectra:

In [ ]:
spectra_dict = cubeviz.specviz.get_spectra(apply_slider_redshift=True)
spectra_dict

This contains the masked spectrum extracted from "Subset 1" region.

In [ ]:
subset1_spec1d = spectra_dict['Spectrum (Subset 1, sum)']
subset1_spec1d